In [2]:
# Import libraries
import pandas as pd
import numpy as np

import fuzzywuzzy
from fuzzywuzzy import process

In [3]:
import charset_normalizer

# set up the Df
countries_df = pd.read_csv('store_income_data_task.csv')
np.random.seed(10)
countries_df.head()

,id,store_name,store_email,department,income,date_measured,country
0,1,"Cullen/Frost Bankers, Inc.",NaN,Clothing,$54438554.24,4-2-2006,United States/
1,2,Nordson Corporation,NaN,Tools,$41744177.01,4-1-2006,Britain
2,3,"Stag Industrial, Inc.",NaN,Beauty,$36152340.34,12-9-2003,United States
3,4,FIRST REPUBLIC BANK,ecanadine3@fc2.com,Automotive,$8928350.04,8-5-2006,Britain/
4,5,Mercantile Bank Corporation,NaN,Baby,$33552742.32,21-1-1973,United Kingdom


In [4]:
# Print the unique countries
countries = countries_df['country'].unique()
print(f"There are {len(countries)} unique countries")
countries

There are 77 unique countries


array(['United States/', 'Britain', ' United States', 'Britain/',
       ' United Kingdom', 'U.K.', 'SA ', 'U.K/', 'America',
       'United Kingdom', nan, 'united states', ' S.A.', 'England ', 'UK',
       'S.A./', 'ENGLAND', 'BRITAIN', 'U.K', 'U.K ', 'America/', 'SA.',
       'S.A. ', 'u.k', 'uk', ' ', 'UK.', 'England/', 'england',
       ' Britain', 'united states of america', 'UK/', 'SA/', 'SA',
       'England.', 'UNITED KINGDOM', 'America.', 'S.A..', 's.a.', ' U.K',
       ' United States of America', 'Britain ', 'England', ' SA',
       'United States of America.', 'United States of America/',
       'United States.', 's. africasouth africa', ' England',
       'United Kingdom ', 'United States of America ', ' UK',
       'united kingdom', 'AMERICA', 'America ',
       'UNITED STATES OF AMERICA', ' S. AfricaSouth Africa', 'america',
       'S. AFRICASOUTH AFRICA', 'Britain.', '/', 'United Kingdom.',
       'United States', ' America', 'UNITED STATES', 'sa',
       'United States

In [5]:
# Function to clean the country list and return unique cleaned values
def clean_country_column(df, column_name):
    df[column_name] = (df[column_name]
                       .str.replace(r"[./]", "", regex=True)
                       .str.replace(r"\s+", " ", regex=True)
                       .str.strip()
                       .str.lower())

    unique_countries = df[column_name].unique()
    print(f"There are {len(unique_countries)} unique countries")
    return unique_countries


# Calling the function and checking for unique values
countries = clean_country_column(countries_df, 'country')
countries

There are 11 unique countries


array(['united states', 'britain', 'united kingdom', 'uk', 'sa',
       'america', nan, 'england', '', 'united states of america',
       's africasouth africa'], dtype=object)

In [116]:
# get the top 10 closest matches to "south africa and so on for the rest of the countries"
matches = fuzzywuzzy.process.extract("south africa", countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

# take a look at them
matches

[('s africasouth africa', 75),
 ('united kingdom', 31),
 ('united states', 24),
 ('sa', 14),
 (nan, 13),
 ('', 0)]

In [121]:
# function to replace rows in the provided column of the provided dataframe
# that match the provided string above the provided ratio with the provided string
# Changing the min_ratio the list of the countries will update according our needs
def replace_matches_in_column(df, column, string_to_match, min_ratio=14):
    # get a list of unique strings
    strings = df[column].unique()

    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings,
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] == min_ratio]

    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches
    df.loc[rows_with_matches, column] = string_to_match

    # let us know the function's done
    print("All done!")

In [124]:
# Calling the function
replace_matches_in_column(df=countries_df, column='country', string_to_match="south africa")

# Filling the NaN values with unknown
countries_df.fillna('unknown', inplace=True)

All done!


In [125]:
# Check agin for unique values
countries = countries_df['country'].unique()
print(f"There are {len(countries)} unique countries")
countries


There are 5 unique countries


array(['united states', 'united kingdom', 'south africa', 'unknown', ''],
      dtype=object)